# SDB Benchmark

## Initialise database

If the following cells raise a `ConnexionRefuseError`, it may be because of the Cassandra database didn't finished to initialize.

In [ ]:
import csv, gc, time, random
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

cluster = Cluster(['cassandra'])
session = cluster.connect()

# Create a keyspace and table
session.execute("""CREATE KEYSPACE IF NOT EXISTS benchmarking WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : '1' }""")
session.execute("""CREATE TABLE IF NOT EXISTS benchmarking.tables (BITCOIN_ADDRESS text, ACCOUNT text, 
IP_ADDRESS text, COUNTY text, COUNTRY_CODE text, DATABASE_COLUMN_TYPE text, PRIMARY KEY 
(ACCOUNT))""")


## Import Data

In [ ]:
# Import data
with open("/home/data.csv", 'r') as file:
    reader = csv.DictReader(file)
    insert_query = session.prepare("""
INSERT INTO benchmarking.tables (BITCOIN_ADDRESS, ACCOUNT, IP_ADDRESS, COUNTY, COUNTRY_CODE, DATABASE_COLUMN_TYPE)
VALUES (?, ?, ?, ?, ?, ?)
""")
    cassandra_times_import = []

    for sample in range(100):
        batch = BatchStatement()

        for i in range(1000):
            row = next(reader)
            session.execute(insert_query, (
                row['BITCOIN_ADDRESS'],
                row['ACCOUNT'],
                row['IP_ADDRESS'],
                row['COUNTY'],
                row['COUNTRY_CODE'],
                row['DATABASE_COLUMN_TYPE']
            ))

        gc.collect()
        start_time = time.perf_counter()

        session.execute(batch)

        end_time = time.perf_counter()
        cassandra_times_import.append(end_time - start_time)

## Requests Generation

Generation of randomized queries

In [ ]:
# Generate random requests
simulated_values = {
    'BITCOIN_ADDRESS': lambda: f"'{''.join(random.choices('ABCDEFGHJKLMNPQRSTUVWXYZ123456789', k=32))}'",
    'ACCOUNT': lambda: f"'{random.randint(10000000, 99999999)}'",
    'IP_ADDRESS': lambda: f"'{random.randint(1, 255)}.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(1, 255)}'",
    'COUNTY': lambda: f"'{random.choice(['Buckinghamshire', 'Avon', 'Cambridgeshire', 'Bedfordshire', 'Borders'])}'",
    'COUNTRY_CODE': lambda: f"'{random.choice('ABCDEFGHJKLMNPQRSTUVWXYZ')}{random.choice('ABCDEFGHJKLMNPQRSTUVWXYZ')}'",
    'DATABASE_COLUMN_TYPE': lambda: f"'{random.choice(['float', 'point', 'int', 'serial', 'varchar', 'blob', 'timestamp'])}'"
}
def generate_random_select(table, column_names, withlike=False, ):
    num_columns = random.randint(1, len(column_names))
    selected_columns = random.sample(column_names, num_columns)

    # num_conditions = random.randint(0,3)
    # conditions = []
    # for _ in range(num_conditions):
    #     column = random.choice(column_names)
    #     if withlike :
    #         operator = random.choice(['=', '!=', 'LIKE'])
    #     else :
    #         operator = random.choice(['=', '!='])
    #     value = simulated_values[column]()  # Générer une valeur simulée pour la colonne
    #     if operator == 'LIKE' and column != 'BITCOIN_ADDRESS':  # LIKE uniquement pour des types texte
    #         value = value.strip("'")
    #         value = f"'%{value}%'"
    #     conditions.append(f"{column} {operator} {value}")

    # if num_conditions :
    #     where_clause = f"WHERE {' AND '.join(conditions)}"
    # elif:
    #     where_clause = ""
    where_clause = ""

    select_query = f"SELECT {', '.join(selected_columns)} FROM {table} {where_clause}"
    return select_query

def generate_random_update_query(table_name, column_names, primary_key, set_of_primary_key_values):
    num_updates = random.randint(1,3)
    columns_to_update = random.sample(column_names, num_updates)
    update_values = [f"{col} = {simulated_values[col]()}" for col in columns_to_update]

    primary_key_value = random.choice(set_of_primary_key_values)

    update_query = f"UPDATE {table_name} SET {', '.join(update_values)} WHERE {primary_key} = '{primary_key_value}'"
    return update_query

def generate_random_delete_query(table_name, primary_key, set_of_primary_key_values):
    primary_key_value = random.choice(set_of_primary_key_values)

    # Construct the UPDATE query
    delete_query = f"DELETE FROM {table_name} WHERE {primary_key} = '{primary_key_value}'"
    return delete_query
    
print("OK")

## Queries Benchmarking

### Select

In [7]:
column_names = ['BITCOIN_ADDRESS', 'ACCOUNT', 'IP_ADDRESS', 'COUNTY', 'COUNTRY_CODE', 'DATABASE_COLUMN_TYPE']

select_queries = []
for _ in range(100):
    select_queries.append(generate_random_select("benchmarking.tables", column_names, withlike=False))

# Benchmark SELECT
cassandra_times_select = []
for i in range(100):
    gc.collect()
    start_time = time.perf_counter()

    session.execute(f"{select_queries[i]}")

    end_time = time.perf_counter()
    cassandra_times_select.append(end_time - start_time)

### Update

In [ ]:
column_names = ['BITCOIN_ADDRESS', 'IP_ADDRESS', 'COUNTY', 'COUNTRY_CODE', 'DATABASE_COLUMN_TYPE']

set_of_key = []
rows = session.execute("SELECT ACCOUNT FROM benchmarking.tables")
for a in rows:
    set_of_key.append(a[0])

update_queries = []
for i in range(100):
    update_queries.append(generate_random_update_query("benchmarking.tables", column_names, "ACCOUNT", set_of_key))

# Benchmark UPDATE
for i in range(100):
    gc.collect()
    start_time = time.perf_counter()

    session.execute(f"{update_queries[i]}")

    end_time = time.perf_counter()
    cassandra_times_select.append(end_time - start_time)
    

### Delete

In [ ]:
set_of_key = []
rows = session.execute("SELECT ACCOUNT FROM benchmarking.tables")
for a in rows:
    set_of_key.append(a[0])

delete_queries = []
for i in range(100):
    delete_queries.append(generate_random_delete_query("benchmarking.tables", "ACCOUNT", set_of_key))

# Benchmark UPDATE
for i in range(100):
    gc.collect()
    start_time = time.perf_counter()

    session.execute(f"{delete_queries[i]}")

    end_time = time.perf_counter()
    cassandra_times_select.append(end_time - start_time)

## Index utilisation

**Creation of an index**

`COUNTY` colum is selected to be indexed because it have a moderate cardinality. The indexing must have a significant effect on queries performances. 

In [ ]:
# Create two tables to compare 
session.execute("""CREATE TABLE IF NOT EXISTS benchmarking.indexedtable (BITCOIN_ADDRESS text, ACCOUNT text, 
IP_ADDRESS text, COUNTY text, COUNTRY_CODE text, DATABASE_COLUMN_TYPE text, PRIMARY KEY 
(ACCOUNT))""")
session.execute("""CREATE TABLE IF NOT EXISTS benchmarking.notindexedtable (BITCOIN_ADDRESS text, ACCOUNT text, 
IP_ADDRESS text, COUNTY text, COUNTRY_CODE text, DATABASE_COLUMN_TYPE text, PRIMARY KEY 
(ACCOUNT))""")

# Import data
with open("/home/data.csv", 'r') as file:
    reader = csv.DictReader(file)
    indexed_insert_query = session.prepare("""
INSERT INTO benchmarking.indexedtable (BITCOIN_ADDRESS, ACCOUNT, IP_ADDRESS, COUNTY, COUNTRY_CODE, DATABASE_COLUMN_TYPE)
VALUES (?, ?, ?, ?, ?, ?)
""")
    notindexed_insert_query = session.prepare("""
INSERT INTO benchmarking.notindexedtable (BITCOIN_ADDRESS, ACCOUNT, IP_ADDRESS, COUNTY, COUNTRY_CODE, DATABASE_COLUMN_TYPE)
VALUES (?, ?, ?, ?, ?, ?)
""")

    batch = BatchStatement()

    for row in reader:
        session.execute(indexed_insert_query, (
            row['BITCOIN_ADDRESS'],
            row['ACCOUNT'],
            row['IP_ADDRESS'],
            row['COUNTY'],
            row['COUNTRY_CODE'],
            row['DATABASE_COLUMN_TYPE']
        ))
        session.execute(notindexed_insert_query, (
            row['BITCOIN_ADDRESS'],
            row['ACCOUNT'],
            row['IP_ADDRESS'],
            row['COUNTY'],
            row['COUNTRY_CODE'],
            row['DATABASE_COLUMN_TYPE']
        ))

    session.execute(batch)

# Create index
create_index_query = """CREATE INDEX contryindex on benchmarking.indexedtable (COUNTY)"""
session.execute(create_index_query)

**Sample of query**

In [ ]:
# With index
gc.collect()
start_time = time.perf_counter()
    
session.execute("SELECT * FROM benchmarking.indexedtable WHERE COUNTY = 'Bedfo'")

end_time = time.perf_counter()
print("With index : ", end_time - start_time)

# Without index
gc.collect()
start_time = time.perf_counter()
    
session.execute("SELECT * FROM benchmarking.notindexedtable WHERE COUNTY = 'Bedfo' ALLOW FILTERING")

end_time = time.perf_counter()
print("Without index : ", end_time - start_time)